# Planning the production of bicycles
---
### 현재 2022년 12월 31일

* 월별 판매 예측 (단위: 천 대)
|Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec|
|---:|---:|---:|---:|---:|---:|---:|---:|---:|---:|---:|---:|
|30|15|15|25|33|40|45|45|26|14|25|30|
* 생산가능량 월 30,000 대
* 초과근무로 생산량 50% 증가 가능. 단, 생산단가 32＄에서 40＄로 증가
* 기초재고 2000대
* 재고비용 월별 대당 5＄ 발생
* 창고 capa 무한
---
### **판매 예측량을 충족**하고 **비용(생산비용 + 재고비용)을 최소화** 하는 월별 생산량, 재고량 산출
---

In [13]:
from ortools.linear_solver import pywraplp

In [14]:
solver = pywraplp.Solver.CreateSolver('SCIP')
infinity = solver.infinity()

### 변수
#### Data
- MONTHS : month index
- DEM : 월별 판매 예측량
- CAP : 월별 생산 capa
- CPROD : 대당 월별 생산 비용
- COVER : 대당 월별 초과 생산 비용
- CSTOCK : 대당 월별 재고 비용
- ISTOCK : 기초재고


In [15]:
MONTHS = ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]
DEM = [30000, 15000, 15000, 25000, 33000, 40000, 45000, 45000, 26000, 14000, 25000, 30000]
CPROD =  32
COVER =  40
CSTOCK =  5
CAP =    30000
ISTOCK =  2000

#### Decision Variable
- $prod_t$ : 월별 생산량
- $over_t$ : 월별 초과 생산량
- $store_t$ : 월별 재고량
- $\forall t \in MONTHS :prod_t, over_t, store_t$ are Integer values
- $\forall t \in MONTHS : prod_t \ge 0, over_t \ge 0, store_t \ge 0$

In [16]:
prod = {}
over = {}
store = {}
for t in range(len(MONTHS)):
    prod[t] = solver.IntVar(0, infinity, 'prod[%i]' % t)
    over[t] = solver.IntVar(0, infinity, 'over[%i]' % t)
    store[t] = solver.IntVar(0, infinity, 'store[%i]' % t)
print('Number of variables =', solver.NumVariables())

Number of variables = 36


#### Constraints
- 생산, 판매, 재고 Balance 제약
    - $prod_1 + over_1 + ISTOCK = DEM_1 + store_1 $
    - $\forall t \in MONTHS, t \ne "Jan" : prod_t + over_t + store_{t-1} = DEM_t + store_t$

In [17]:
for t in range(len(MONTHS)):
    if t == 0 :
        solver.Add(prod[t] + over[t] + ISTOCK >=  DEM[t] + store[t])
    else :
        solver.Add(prod[t] + over[t] + store[t-1] >= DEM[t] + store[t])

- 생산 capa 제약
    - $\forall t \in MONTHS : prod_t \le CAP$
    - $\forall t \in MONTHS : over_t \le 0.5 * CAP$

In [18]:
for t in range(len(MONTHS)):
    solver.Add(prod[t] <= CAP)
    solver.Add(over[t] <= 0.5*CAP)

In [19]:
# 안전재고 제약 : 1000대 이상 안전재고 설정

for t in range(len(MONTHS)):
    solver.Add(store[t] >= 1000)

In [20]:
print('Number of constraints =', solver.NumConstraints())

Number of constraints = 48


#### objective function
- $minimize \sum_{t\in MONTHS}(CPROD * prod_t + COVER * over_t + CSTOCK * store_t)$

In [21]:
obj_exp = 0
for t in range(12) :
    obj_exp = obj_exp + CPROD*prod[t] + COVER*over[t] + CSTOCK*store[t]
solver.Minimize(obj_exp)

In [22]:
solver.Solve()

0

In [23]:
print ("Mon\tDemand\tProd\tOver\tStock")
print ("init\t\t\t\t" + str(ISTOCK))
for t in range(12):
    print(str(MONTHS[t]) + '\t' + str(DEM[t]) + '\t' + str(prod[t].solution_value()) + '\t' + str(over[t].solution_value()) \
          + '\t' + str(store[t].solution_value()) )

Mon	Demand	Prod	Over	Stock
init				2000
Jan	30000	29000.0	0.0	1000.0
Feb	15000	15000.0	0.0	1000.0
Mar	15000	15000.0	0.0	1000.0
Apr	25000	28000.0	0.0	4000.0
May	33000	30000.0	0.0	1000.0
Jun	40000	30000.0	10000.0	1000.0
Jul	45000	30000.0	15000.0	1000.0
Aug	45000	30000.0	15000.0	1000.0
Sep	26000	26000.0	0.0	1000.0
Oct	14000	14000.0	0.0	1000.0
Nov	25000	25000.0	0.0	1000.0
Dec	30000	30000.0	0.0	1000.0


In [24]:
solver.Objective().Value()

11339000.0